In [ ]:
import pyomo.environ as pyo
import pandas as pd
import math


clientes_df = pd.read_csv("clients.csv")   # columnas: ClientID, Latitude, Longitude, Demand
depot_df    = pd.read_csv("depots.csv")      # columnas: DepotID, Latitude, Longitude
veh_df      = pd.read_csv("vehicles.csv")  # columnas: VehicleID, Capacity, Range
params_df   = pd.read_csv("parameters_base.csv") # columnas: Parameter, Value
DEPOT = 0
depot_lat  = depot_df["Latitude"].iloc[0]
depot_lon  = depot_df["Longitude"].iloc[0]
coords = {0: (depot_lat, depot_lon)}
CLIENTES = clientes_df["ClientID"].tolist()

for _, row in clientes_df.iterrows():
    coords[row["ClientID"]] = (row["Latitude"], row["Longitude"])

demand = {row["ClientID"]: row["Demand"] for _, row in clientes_df.iterrows()}

# Vehículos
V = veh_df["VehicleID"].tolist()

rv = {row["VehicleID"]: row["Capacity"] for _, row in veh_df.iterrows()}
tv = {row["VehicleID"]: row["Range"]    for _, row in veh_df.iterrows()}

# Parámetros
fuel_price = params_df.loc[params_df["Parameter"] == "fuel_price", "Value"].iloc[0]
fuel_eff   = params_df.loc[params_df["Parameter"] == "fuel_efficiency_typical", "Value"].iloc[0]

# Convertir eficiencia a "km/galón"
ev = {v: fuel_eff for v in V}

# Costos 
co = 50000     # costo operativo por vehículo
pf = fuel_price     # precio del combustible por litro
# funciones auxiliares
def safe_value(var):
    try:
        return pyo.value(var)
    except:
        return None

def fmt_num(x, fmt="{:,.0f}"):
    return fmt.format(x) if x is not None and x != "" else "N/A"
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c
def compute_cost(dist_km, efficiency, price):
    # galones consumidos = km / (km/galón)
    return price * (dist_km / efficiency)


NODOS = [DEPOT] + CLIENTES
w = {}
for i in NODOS:
    for j in NODOS:
        if i == j:
            w[i, j] = 0.0
        else:
            w[i, j] = haversine_km(coords[i][0], coords[i][1], coords[j][0], coords[j][1])

# Imprimir matriz de distancias para verificar
print("MATRIZ DE DISTANCIAS CREADA AUTOMÁTICAMENTE:")
for i in NODOS:
    print(i, [round(w[i,j],2) for j in NODOS])

# model
model = pyo.ConcreteModel()

# Conjuntos
model.N = pyo.Set(initialize=NODOS)
model.C = pyo.Set(initialize=CLIENTES)
model.V = pyo.Set(initialize=V)
model.D = pyo.Set(initialize=[DEPOT])

# Parámetros
model.w = pyo.Param(model.N, model.N, initialize=w, within=pyo.NonNegativeReals)
model.q = pyo.Param(model.C, initialize=demand, within=pyo.NonNegativeReals)
model.r = pyo.Param(model.V, initialize=rv, within=pyo.NonNegativeReals)
model.tau = pyo.Param(model.V, initialize=tv, within=pyo.NonNegativeReals)
model.e = pyo.Param(model.V, initialize=ev, within=pyo.PositiveReals)

model.co = pyo.Param(initialize=co) 
model.pf = pyo.Param(initialize=pf)

# costo por arco 
def s_init(m, i, j, v):
    dist = m.w[i,j]
    return compute_cost(dist, m.e[v], m.pf)
model.s = pyo.Param(model.N, model.N, model.V, initialize=s_init, within=pyo.NonNegativeReals)

# Variables 
model.x = pyo.Var(model.N, model.N, model.V, domain=pyo.Binary)
model.y = pyo.Var(model.V, domain=pyo.Binary)
model.u = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals, bounds=(0, max(rv.values())))

def no_self_arcs_rule(m, i, v):
    return m.x[i, i, v] == 0
model.no_self_arcs = pyo.Constraint(model.N, model.V, rule=no_self_arcs_rule)

# Función Objetivo
def obj_rule(m):
    return sum(m.s[i, j, v] * m.x[i, j, v] for i in m.N for j in m.N for v in m.V) + sum(m.co * m.y[v] for v in m.V)
model.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Restricciones
def flow_cons_rule(m, v, c):
    return sum(m.x[i, c, v] for i in m.N if i != c) - sum(m.x[c, j, v] for j in m.N if j != c) == 0
model.flow_cons = pyo.Constraint(model.V, model.C, rule=flow_cons_rule)

def autonomy_rule(m, v):
    return sum(m.x[i, j, v] * m.w[i, j] for i in m.N for j in m.N if i != j) <= m.tau[v]
model.autonomy = pyo.Constraint(model.V, rule=autonomy_rule)

def mtz_rule(m, c, j, v):
    if c == j:
        return pyo.Constraint.Skip
    return m.u[c, v] - m.u[j, v] + m.r[v] * m.x[c, j, v] <= m.r[v] - m.q[j]
model.mtz = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

def u_lower_rule(m, c, v):
    return m.q[c] <= m.u[c, v]
model.u_lower = pyo.Constraint(model.C, model.V, rule=u_lower_rule)

def u_upper_rule(m, c, v):
    return m.u[c, v] <= m.r[v]
model.u_upper = pyo.Constraint(model.C, model.V, rule=u_upper_rule)

def depart_from_depot_rule(m, v):
    return sum(m.x[DEPOT, j, v] for j in m.C) == m.y[v]
model.depart_depot = pyo.Constraint(model.V, rule=depart_from_depot_rule)

def return_to_depot_rule(m, v):
    return sum(m.x[i, DEPOT, v] for i in m.C) == m.y[v]
model.return_depot = pyo.Constraint(model.V, rule=return_to_depot_rule)

def cover_once_rule(m, j):
    return sum(m.x[i, j, v] for v in m.V for i in m.N if i != j) == 1
model.cover_once = pyo.Constraint(model.C, rule=cover_once_rule)

solver = pyo.SolverFactory('glpk')
result = solver.solve(model, tee=False)

# Calcular costos
costo_variable = 0
costo_fijo = 0
rutas_vehiculos = {}

for v in model.V:
    y_val = pyo.value(model.y[v])
    if y_val > 0.5:  # Vehículo utilizado
        costo_fijo += co
        
        # Reconstruir ruta
        route = ["CD0"]
        current = DEPOT
        visited = set()
        
        for _ in range(len(NODOS) + 2):
            found_next = False
            for j in NODOS:
                if j != current:
                    x_val = pyo.value(model.x[current, j, v])
                    if x_val is not None and x_val > 0.5:
                        if j == DEPOT:
                            route.append("CD0")
                            current = j
                            found_next = True
                            break
                        else:
                            route.append(f"C{j}")
                            visited.add(j)
                            current = j
                            found_next = True
                            break
            if not found_next or current == DEPOT:
                break
        
        # Calcular métricas de la ruta
        distancia_ruta = 0
        carga_ruta = 0
        
        for i in range(len(route) - 1):
            node_i = DEPOT if route[i] == "CD0" else int(route[i][1:])
            node_j = DEPOT if route[i + 1] == "CD0" else int(route[i + 1][1:])
            distancia_ruta += w[node_i, node_j]
            
            # Sumar costo variable para este arco
            s_val = (0 + pf/ev[v]) * w[node_i, node_j]
            costo_variable += s_val
        
        for client in visited:
            carga_ruta += demand[client]
        
        rutas_vehiculos[v] = {
            'ruta': route,
            'distancia': distancia_ruta,
            'carga': carga_ruta,
            'clientes': visited
        }

# mostrasr rtsults
costo_total = costo_variable + costo_fijo
print("Resumen")
print(f"Costo variable: {costo_variable:,.0f} COP")
print(f"Costo fijo: {costo_fijo:,.0f} COP")
print(f"Costo total: {costo_total:,.0f} COP")

# Mostrar rutas optimizadas
print("Rutas")
for v, datos in rutas_vehiculos.items():
    print(f"\nVehículo {v} (Capacidad: {rv[v]}kg, Autonomia: {tv[v]}km):")
    print(f"  Ruta: {' → '.join(datos['ruta'])}")
    print(f"  Distancia: {datos['distancia']:.1f} km / {tv[v]} km")
    print(f"  Carga: {datos['carga']} kg / {rv[v]} kg")
    print(f"  Clientes: {sorted([f'C{c}' for c in datos['clientes']])}")

# Validación 
print(f"Total demanda: {sum(demand.values())} kg")


print("Resulktados")

for v, datos in rutas_vehiculos.items():
    print(f"\nVehículo {v}:")
    print(f"Ruta completa: {' → '.join(datos['ruta'])}")
    
    #  distancia tramo por tramo
    distancia_detallada = 0
    costo_vehiculo = 0
    for i in range(len(datos['ruta']) - 1):
        node_i = DEPOT if datos['ruta'][i] == "CD0" else int(datos['ruta'][i][1:])
        node_j = DEPOT if datos['ruta'][i + 1] == "CD0" else int(datos['ruta'][i + 1][1:])
        tramo_distancia = w[node_i, node_j]
        tramo_costo = (0 + pf/ev[v]) * tramo_distancia
        
        distancia_detallada += tramo_distancia
        costo_vehiculo += tramo_costo
        
        print(f"  {datos['ruta'][i]} → {datos['ruta'][i+1]}: {tramo_distancia} km × {0 + pf/ev[v]:.0f} COP/km = {tramo_costo:,.0f} COP")
    
    print(f"Distancia total: {distancia_detallada} km")
    print(f"Costo variable vehículo {v}: {costo_vehiculo:,.0f} COP")

MATRIZ DE DISTANCIAS CREADA AUTOMÁTICAMENTE:
0 [0.0, 17.26, 10.62, 6.38, 16.6, 10.62, 9.73, 15.43, 10.65, 15.24, 21.83, 17.09, 5.94, 12.49, 4.0, 16.81, 21.21, 14.53, 10.19, 12.21, 3.53, 14.3, 15.05, 4.31, 18.69]
1 [17.26, 0.0, 10.32, 12.44, 0.81, 9.2, 7.54, 13.96, 9.92, 6.41, 4.61, 8.79, 17.62, 5.37, 14.45, 0.86, 4.72, 3.48, 7.09, 13.21, 16.73, 18.46, 3.34, 14.97, 3.79]
2 [10.62, 10.32, 0.0, 4.24, 9.52, 10.8, 5.77, 6.02, 11.52, 12.72, 14.6, 15.6, 8.15, 5.2, 6.66, 9.53, 15.04, 9.67, 6.46, 3.5, 8.31, 8.64, 7.05, 6.64, 13.49]
3 [6.38, 12.44, 4.24, 0.0, 11.67, 9.31, 5.48, 9.51, 9.82, 12.7, 17.02, 15.29, 5.25, 7.13, 2.46, 11.8, 16.93, 10.59, 6.21, 6.35, 4.44, 10.03, 9.61, 2.65, 14.83]
4 [16.6, 0.81, 9.52, 11.67, 0.0, 8.89, 6.87, 13.23, 9.65, 6.53, 5.35, 9.07, 16.84, 4.58, 13.72, 0.34, 5.53, 3.26, 6.46, 12.41, 15.98, 17.68, 2.55, 14.21, 4.44]
5 [10.62, 9.2, 10.8, 9.31, 8.89, 0.0, 5.09, 16.71, 0.83, 4.78, 13.05, 6.5, 13.89, 7.51, 9.83, 9.22, 11.73, 5.73, 4.53, 14.24, 11.9, 18.97, 9.0, 10.65, 